# 1.Yolov5的数据准备

主要是在指定的路径下放好训练所需的数据

In [1]:
ENV = "Meg"
datadir = './tmp'
outputdir = '.'
archivedir = './archive'

In [2]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch
import time
from IPython.display import Image, clear_output  # to display images
from os import listdir
from os.path import isfile, join
import yaml
from sklearn.model_selection import GroupKFold


In [3]:
class CFG:
    suffix = "2400"
    batch_size = 64
    epochs = 50
    image_size = 672 #记得改文件名  siim_yolo_label_txt
    model_pt = f"{outputdir}/yolov5-master/models/yolov5_kaggle.yaml"
    
    debug = False

In [6]:
# 切分fold
train_df = pd.read_csv(f'{datadir}/train_image_level.csv')
fold = 0
gkf  = GroupKFold(n_splits = 5)
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df, groups=train_df.StudyInstanceUID.tolist())):
    train_df.loc[val_idx, 'fold'] = fold

train_df['image_path'] = f'{datadir}/train/image/' + train_df["id"]+ '.png'
val_files   = list(train_df[train_df.fold==fold].image_path.unique())
train_files = list(train_df[train_df.fold!=fold].image_path.unique())
len(train_files), len(val_files)
train_df

,id,boxes,label,StudyInstanceUID,fold,image_path
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,4,./tmp//train/image/000a312787f2_image.png
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,0,./tmp//train/image/000c3a3f293f_image.png
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,4,./tmp//train/image/0012ff7358bc_image.png
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,1,./tmp//train/image/001398f4ff4f_image.png
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,3,./tmp//train/image/001bd15d1891_image.png
...,...,...,...,...,...,...
6329,ffcc6edd9445_image,NaN,none 1 0 0 1 1,7e6c68462e06,3,./tmp//train/image/ffcc6edd9445_image.png
6330,ffd91a2c4ca0_image,NaN,none 1 0 0 1 1,8332bdaddb6e,3,./tmp//train/image/ffd91a2c4ca0_image.png
6331,ffd9b6cf2961_image,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,2,./tmp//train/image/ffd9b6cf2961_image.png
6332,ffdc682f7680_image,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d,1,./tmp//train/image/ffdc682f7680_image.png


In [7]:
# 将含label的txt文件，划分到train和val文件夹下.
label_dir = f'{datadir}/siim_yolo_label_txt'
os.makedirs(f'{datadir}/siim_yolo_dataset/labels/train', exist_ok = True)
os.makedirs(f'{datadir}/siim_yolo_dataset/labels/val', exist_ok = True)
os.makedirs(f'{datadir}/siim_yolo_dataset/images/train', exist_ok = True)
os.makedirs(f'{datadir}/siim_yolo_dataset/images/val', exist_ok = True)
for file in tqdm(train_files):
    shutil.copy(file, f'{datadir}/siim_yolo_dataset/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), f'{datadir}/siim_yolo_dataset/labels/train')
    
for file in tqdm(val_files):
    shutil.copy(file, f'{datadir}/siim_yolo_dataset/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), f'{datadir}/siim_yolo_dataset/labels/val')

  0%|          | 0/5068 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: './tmp//siim_yolo_label_txt\\000c3a3f293f_image.txt'

In [ ]:
# 新建一个data yaml文件
classes = ['0. opacity']

with open(join( datadir , 'siim_yolo_dataset/train.txt'), 'w') as f:
    for path in glob(f'{datadir}/siim_yolo_dataset/images/train/*'):
        f.write(path+'\n')
            
with open(join( datadir , 'siim_yolo_dataset/val.txt'), 'w') as f:
    for path in glob(f'{datadir}/siim_yolo_dataset/images/val/*'):
        f.write(path+'\n')

data = dict(
    train =  join( datadir , 'siim_yolo_dataset/train.txt'),
    val   =  join( datadir , 'siim_yolo_dataset/val.txt' ),
    nc    = 1,
    names = classes
    )

with open(join( datadir , 'siim_yolo_dataset/siim_dataset.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( datadir , 'siim_yolo_dataset/siim_dataset.yaml'), 'r')
print(f.read())

# 2.Yolov5的训练

In [1]:
%cd workspace/yolov5-master

[WinError 3] 系统找不到指定的路径。: 'workspace/yolov5'
C:\Users\xm\jupyternotebook


In [ ]:
!WANDB_MODE="dryrun" python train.py --img $CFG.image_size --batch-size $CFG.batch_size --worker 16 --epochs $CFG.epochs --data /home/megstudio/workspace/competedata/siim_yolo_dataset/siim_dataset.yaml --hyp /home/megstudio/workspace/hyp.scratch.yaml --cfg $CFG.model_pt --weights '' --cache